# Task description

Word Sense Induction (WSI) is the process of automatic identification of the word
senses. 

The goal of this task is to use methods of distributional semantics and word embeddings to solve word sense induction. 

You can obtain additional information at the web site of the competition https://russe.nlpub.org/2018/wsi/ and https://competitions.codalab.org/competitions/27331#learn_the_details .

In this notebook we consider obtained solution on wiki-wiki dataset.

# Installation

Install requirements and upload github repo on google disk

In [ ]:
!pip install pymorphy2
!pip install tensorflow-hub==0.7.0
!pip install tensorflow==1.15.2
!pip install deeppavlov
!git clone https://github.com/nlpub/russe-wsi-kit.git

fatal: destination path 'russe-wsi-kit' already exists and is not an empty directory.


Restart the environment after installation of libraries

# Import libraries

In [ ]:
from deeppavlov import build_model
from deeppavlov import configs
from deeppavlov.core.common.file import read_json
from deeppavlov.core.commands.infer import build_model

# Load pretrained model

**Try pretrained models:**



*   ELMO model pretrained on wiki gives the highest score on wiki dataset. ARI score is 0.65;
*   ELMO model pretrained on news gives very low score;
*   Bert_sentence_embedder gives ARI score  0.6169.
*   Baseline adagram model gives ARI score 0.6278

Finally, choose ELMO model, pretrained on wikipedia  dataset from DeepPavlov library.

https://github.com/deepmipt/DeepPavlov

In [ ]:
# build pretrained ELMO model
faq = build_model(configs.embedder.elmo_ru_wiki, download = True)

2020-11-21 07:37:53.255 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-wiki_600k_steps.tar.gz to /root/.deeppavlov/downloads/embeddings/elmo_ru-wiki_600k_steps.tar.gz
100%|██████████| 694M/694M [02:00<00:00, 5.78MB/s]
2020-11-21 07:39:53.630 INFO in 'deeppavlov.core.data.utils'['utils'] at line 268: Extracting /root/.deeppavlov/downloads/embeddings/elmo_ru-wiki_600k_steps.tar.gz archive into /root/.deeppavlov/downloads/embeddings/elmo_ru_wiki
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes




INFO:tensorflow:Saver not created because there are no variables in the graph to restore



# Data loading

Test our model on wiki dataset

In [ ]:
data_path = 'russe-wsi-kit/data/main/wiki-wiki/test.csv'

In [ ]:
import pandas as pd
df = pd.read_csv(data_path, sep='\t')

In [ ]:
df

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,2074,давление,NaN,NaN,0-9,Давление пара создается движением поршня в цил...
1,2075,давление,NaN,NaN,13-22,"«У тебя что, давление поднялось?» Я сказал, чт..."
2,2076,давление,NaN,NaN,56-65,Я жалуюсь Никоновичу наконец на головокружение...
3,2077,давление,NaN,NaN,0-9,Давление в котле не менялось
4,2078,давление,NaN,NaN,25-34,Он каждые два часа мерил давление и сахар в крови
...,...,...,...,...,...,...
3724,5798,зуд,NaN,NaN,43-47,Многих американцев одолевает романтический зуд...
3725,5799,зуд,NaN,NaN,23-27,Если на нее не находил зуд рассказывания истор...
3726,5800,зуд,NaN,NaN,27-33,"С раздражающей завистью, с зудом неудовлетворе..."
3727,5801,зуд,NaN,NaN,12-16,Нестерпимый зуд любопытства


# **Preprocessing**

**Preprocess context of words:** 


*   split text into words;
*   lemmatize;
*   lowercase;
*   remove stopwords;
*   remove punctuation;
*  remove foreign words;
*  remove numbers;
*  remove prepositions, pronouns and other not so important words.

Using not all preprocessing steps didn't improve the result.



In [ ]:
import pymorphy2
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import string
import itertools

In [ ]:
def pos(word, morth=pymorphy2.MorphAnalyzer()):
    "Return a likely part of speech for the *word*."""
    return morth.parse(word)[0].tag.POS

In [ ]:
stop_words = nltk.download('stopwords')
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words('russian')

words = [] #tokens of sentence
embs = []  #embedings 

for j in tqdm(range(df.shape[0])):
  emb = [morph.parse(word)[0].normal_form.lower() for word in re.findall(r'[А-я]+', df['context'][j])]
  token = [i for i in emb if ( i not in stop_words)]
  functors_pos = {'INTJ','NPRO', 'PRCL', 'CONJ', 'PREP'}  # function words
  new_token = [word for word in token if pos(word) not in functors_pos] 
  words.append(new_token)
  embs.append(faq([' '.join(token)])[0])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 3729/3729 [20:24<00:00,  3.05it/s]


# Build text embeddings

From pretrained contextual embeddings for words build embedding representations for sentences.

**Try different approaches:**



*   Find target words in sentence and then average their vectors. G
*   Average all words in sentence. 
*   Average neighbours to target words vectors and target vectors in sentence.
*   Average k-nearest to target word vectors. Try k = 5 and k = 3. 

Get following results:


\begin{array}{ccc}
\text{method}&\text{ARI score}\\
avg\_target\_words& 0.652\\
avg\_all\_words&  0.652\\
avg\_neighbours & 	0.616\\
3-nearest\_neighbours & 0.59\\
5-nearest\_neighbours& 0.616 
\end{array}




Finally decided to use approach with averaging all words in sentence.


In [ ]:
from sklearn.neighbors import NearestNeighbors

def add_indexes_of_neighbours(indexes_of_word, num_words_context):
  # function for including in embedding neighbour words with the same weight
  new_indexes = []
  for i in indexes_of_word:
    if (i==0  and  num_words_context>2): #target word is first
      # add only neighbours from right
      new_indexes.append([1,2])
    elif (i==1 and  num_words_context>2):
      new_indexes.append([0, 2])
    elif (i == num_words_context-1):
      new_indexes.append([i-1, i-2])
    elif (i ==  num_words_context-2):
      new_indexes.append([i-1, i+1])
    else:
      new_indexes.append([i-1, i+1])
  
  final_list = indexes_of_word + list(itertools.chain(*new_indexes))
  return list(set(final_list))

def k_nearest_neighbours(word, embs, k=5):
  """ for each word search in sentence k-nearest words and returns its indexes"""
  samples = embs
  neigh = NearestNeighbors(n_neighbors=k)
  neigh.fit(samples)
  closest_words_idx = neigh.kneighbors(faq([morph.parse(word)[0].normal_form.lower()])[0])[1]
  return list(itertools.chain(*closest_words_idx.tolist()))


In [ ]:
import numpy as np
from more_itertools import locate
import itertools

context_embs = []
for z in tqdm(range(df.shape[0])):
#   # use nearest neighbours
  # if len(embs[z])>15:
  #   indexes_of_word = k_nearest_neighbours(df.word[z], embs[z], k=10)
  #   context_embs.append(list(embs[z][indexes_of_word,:].mean(axis=0))) #average only target words in context
  # elif len(embs[z])>6:
  #   indexes_of_word = k_nearest_neighbours(df.word[z], embs[z], k=5)
  #   context_embs.append(list(embs[z][indexes_of_word,:].mean(axis=0))) #average only target words in context
  # else:
  #   indexes_of_word = k_nearest_neighbours(df.word[z], embs[z], k=1)
  #   context_embs.append(list(embs[z][indexes_of_word,:].mean(axis=0)))
  context_embs.append(list(embs[z].mean(axis=0)))    # average words in full sentence

100%|██████████| 3729/3729 [09:02<00:00,  6.87it/s]


In [ ]:
# define indexes for each unique word
unique_words = df.word.unique()
# find indexes of unique words
indexes = [list(df.word).index(unique_words[i]) for i in range(unique_words.shape[0])] + [df.shape[0]]

# Number of clusters

Choose the number of clusters from 2 to 5, using silhouette scores.

In [ ]:
from sklearn.metrics import silhouette_score 
def define_num_clusters(X_array):
  algs = [AgglomerativeClustering(n_clusters=i) for i in range(2, 6, 1)]
  k = [2, 3, 4, 5, 6, 7] 
  # Appending the silhouette scores of the different models to the list 
  silhouette_scores = [] 
  for j in range(4):
    silhouette_scores.append(silhouette_score(X_array, algs[j].fit_predict(X_array)))
  return k[np.argmax(silhouette_scores)]

# Clusterization approaches

**Test different clusterization approaches**



1.  Agglomerating clusterization gives the highest score;
2. Affinity propogation gives very low score;
3.   KMeans clusterization gives the same score as Agglomerative clustering.

Finally decided to use Agglomerative clustering with number of clusters, defined with silhouette scores,  cosine affinity and use average linkage.



In [ ]:
from sklearn.cluster import AgglomerativeClustering
clusters = []
for i in tqdm(range(unique_words.shape[0])):
  if np.array(context_embs)[indexes[i]:(indexes[i+1])].shape[0]< 7:
    num_clusters = 2
  else:
    num_clusters = define_num_clusters(np.array(context_embs)[indexes[i]:(indexes[i+1])])
  print(num_clusters)
  clusters.append(list(AgglomerativeClustering(n_clusters=num_clusters).fit(np.array(context_embs)[indexes[i]:(indexes[i+1])]).labels_))

  1%|          | 1/168 [00:02<06:36,  2.37s/it]

2
3


  2%|▏         | 3/168 [00:07<06:32,  2.38s/it]

3


  2%|▏         | 4/168 [00:09<06:30,  2.38s/it]

...

2


 99%|█████████▉| 167/168 [06:37<00:02,  2.36s/it]

4


100%|██████████| 168/168 [06:39<00:00,  2.38s/it]

2


In [ ]:
# from sklearn.cluster import AffinityPropagation
# clusters = []
# for i in range(unique_words.shape[0]):
#   clusters.append(list(AffinityPropagation().fit(np.array(context_embs)[indexes[i]:(indexes[i+1])]).labels_))

# from sklearn.cluster import KMeans
# clusters = []
# for i in range(unique_words.shape[0]):
#   clusters.append(list(KMeans(n_clusters=2).fit(np.array(context_embs)[indexes[i]:(indexes[i+1])]).labels_))

## Final results 

In [ ]:
predict_list = list(itertools.chain(*clusters))
df['predict_sense_id'] = predict_list

# Save

In [ ]:
save_path = 'russe-wsi-kit/data/main/wiki-wiki/elmo.csv'
df.to_csv(save_path, sep='\t', index=None)

In [ ]:
cd russe-wsi-kit/data/main/wiki-wiki/

/content/russe-wsi-kit/data/main/active-dict


In [ ]:
!zip wiki-wiki.zip elmo.csv

  adding: f_elmo.csv (deflated 69%)


In [ ]:
cd /content/

/content


# Test score

In [ ]:
from sklearn.metrics import adjusted_rand_score
score = []
for j in range(unique_words.shape[0]):
  score.append(adjusted_rand_score(clusters[j], np.array(list(df['gold_sense_id'])[indexes[j]:(indexes[j+1])])))
print(score)